<a href="https://colab.research.google.com/github/neekBruh/PrimerParcialIA2024/blob/main/Grommeck_1P_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>